[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/emiletimothy/Caltech-CS155-2023/blob/main/set5/set5_prob2.ipynb)


# Problem 2

In [29]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

In [30]:
movies = pd.read_csv("../movies.csv")
data = pd.read_csv("../data.csv")
print(data)

        User ID  Movie ID  Rating
0             0        88     4.0
1             0       545     3.5
2             0         7     3.0
3             0       401     4.0
4             0      1230     0.5
...         ...       ...     ...
121496      991       681     1.0
121497      991       736     3.5
121498      991       162     3.0
121499      991      1242     3.5
121500      991       321     4.5

[121501 rows x 3 columns]


In [31]:
Y = np.array(data)
print(Y)
print(len(Y))
random.shuffle(Y)
print(Y)

[[   0.    88.     4. ]
 [   0.   545.     3.5]
 [   0.     7.     3. ]
 ...
 [ 991.   162.     3. ]
 [ 991.  1242.     3.5]
 [ 991.   321.     4.5]]
121501
[[   0.    88.     4. ]
 [   0.   545.     3.5]
 [   0.    88.     4. ]
 ...
 [ 712.    57.     5. ]
 [ 727.  1492.     3.5]
 [ 864.   193.     3.5]]


In [41]:
from numpy.random.mtrand import gamma
def grad_U(Ui, Yij, Vj, Ai, Bj, reg, eta):
    """
    Takes as input Ui (the ith row of U), a training point Yij, the column
    vector Vj (jth column of V^T), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Ui multiplied by eta.
    """
    sum = Vj*(Yij - (np.dot(Ui, Vj) + Ai + Bj))
    grad = reg*Ui-sum
    return eta*grad

def grad_V(Vj, Yij, Ui, Ai, Bj, reg, eta):
    """
    Takes as input the column vector Vj (jth column of V^T), a training point Yij,
    Ui (the ith row of U), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Vj multiplied by eta.
    """
    sum = Ui*(Yij - (np.dot(Ui, Vj) + Ai + Bj))
    grad = reg*Vj-sum
    return eta*grad

def grad_A(Vj, Yij, Ui, Ai, Bj, eta): ############ NEW
    """
    Takes as input the column vector Vj (jth column of V^T), a training point Yij,
    Ui (the ith row of U), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Vj multiplied by eta.
    """
    grad = -(Yij - (np.dot(Ui, Vj) + Ai + Bj))
    return eta*grad

def grad_B(Vj, Yij, Ui, Ai, Bj, eta): ############ NEW
    """
    Takes as input the column vector Vj (jth column of V^T), a training point Yij,
    Ui (the ith row of U), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Vj multiplied by eta.
    """
    grad = -(Yij - (np.dot(Ui, Vj) + Ai + Bj))
    return eta*grad

def get_err(A, B, U, V, Y, reg=0.0):
    """
    Takes as input a matrix Y of triples (i, j, Y_ij) where i is the index of a user,
    j is the index of a movie, and Y_ij is user i's rating of movie j and
    user/movie matrices U and V.

    Returns the mean regularized squared-error of predictions made by
    estimating Y_{ij} as the dot product of the ith row of U and the jth column of V^T.
    """
    sum = 0
    for y in Y:
      i = int(y[0])
      j = int(y[1])
      Yij = y[2]
      # print("i: "+str(i)+" j: "+str(j))
      # print("len U: "+str(len(U))+" len V: "+str(len(V)))
      sum += (Yij-(np.dot(U[i], V[j]) + A[i] + B[j]))**2
      
    return float(sum/(2*len(Y)))



def train_model(M, N, K, eta, reg, Y, eps=0.0001, max_epochs=300):
    """
    Given a training data matrix Y containing rows (i, j, Y_ij)
    where Y_ij is user i's rating on movie j, learns an
    M x K matrix U and N x K matrix V such that rating Y_ij is approximated
    by (UV^T)_ij.

    Uses a learning rate of <eta> and regularization of <reg>. Stops after
    <max_epochs> epochs, or once the magnitude of the decrease in regularized
    MSE between epochs is smaller than a fraction <eps> of the decrease in
    MSE after the first epoch.

    Returns a tuple (U, V, err) consisting of U, V, and the unregularized MSE
    of the model.
    """
    U = np.random.uniform(low=-0.5, high=0.5, size=(M,K))
    V = np.random.uniform(low=-0.5, high=0.5, size=(N,K))
    A = np.random.uniform(low=-0.5, high=0.5, size=(M))
    B = np.random.uniform(low=-0.5, high=0.5, size=(N))
    e=0
    mse_0 = get_err(A, B, U, V, Y, reg)
    prev_mse = mse_0
    cur_mse = mse_0
    first_mse = 0
    first = True
    while e<max_epochs:
      np.random.shuffle(Y)
      for y in Y:
          #we tryna minimize error of UV^T and Y
        i = int(y[0]) #uxer
        j = int(y[1]) #movie
        Yij = y[2] #user's rating

        #build U and V using gradient descent
        ##### wouold we update the A and B here??
          #update the values for a specific user's movie pref params
        U[i] -= grad_U(U[i], Yij, V[j], A[i], B[j], reg, eta) 
          #update the values for a specific movie's relationships to users
        V[j] -= grad_V(V[j], Yij, U[i], A[i], B[j], reg, eta)

        A -= grad_A(V[j], Yij, U[i], A[i], B[j], eta) 
        B -= grad_B(V[j], Yij, U[i], A[i], B[j], eta)

      #obtain the error
      cur_mse = get_err(A, B, U, V, Y, reg)
      if(first):
        first_mse = mse_0-cur_mse
        first = False
      #stop when change in error gets too small
      if (prev_mse-cur_mse)<=(first_mse*eps):
        break

      prev_mse = cur_mse
      e += 1

    return U, V, A, B, cur_mse

In [42]:
Y_train = Y[0:109351]
Y_test = Y[109351:121501]

In [43]:

M = max(max(Y_train[:,0]), max(Y_test[:,0])).astype(int) # users
N = max(max(Y_train[:,1]), max(Y_test[:,1])).astype(int) # movies

M += 1
N += 1
print("Factorizing with ", M, " users, ", N, " movies.")
K = 20

reg = 0.0
eta = 0.03 # learning rate
E_in = []
E_out = []

# Use to compute Ein and Eout

U,V, A,B, err = train_model(M, N, K, eta, reg, Y_train)


Factorizing with  992  users,  1500  movies.


In [44]:
# print(U)
# print(V)
# print(len(Y_test))
# print(Y_test)
print(f"Testing error: {get_err(A, B, U, V, Y_test)}")

Testing error: 0.3990683537970572


## 2E:
Run the cell below to get your graphs. This might take a long time to run, but it should take less than 2 hours. I would encourage you to validate your 2C is correct.

In [ ]:
Y_train = np.loadtxt('train.txt').astype(int)
Y_test = np.loadtxt('test.txt').astype(int)

M = max(max(Y_train[:,0]), max(Y_test[:,0])).astype(int) # users
N = max(max(Y_train[:,1]), max(Y_test[:,1])).astype(int) # movies
Ks = [10,20,30,50,100]

regs = [10**-4, 10**-3, 10**-2, 10**-1, 1]
eta = 0.03 # learning rate
E_ins = []
E_outs = []

# Use to compute Ein and Eout
for reg in regs:
    E_ins_for_lambda = []
    E_outs_for_lambda = []

    for k in Ks:
        print("Training model with M = %s, N = %s, k = %s, eta = %s, reg = %s"%(M, N, k, eta, reg))
        U,V, e_in = train_model(M, N, k, eta, reg, Y_train)
        E_ins_for_lambda.append(e_in)
        eout = get_err(U, V, Y_test)
        E_outs_for_lambda.append(eout)

    E_ins.append(E_ins_for_lambda)
    E_outs.append(E_outs_for_lambda)


# Plot values of E_in across k for each value of lambda
for i in range(len(regs)):
    plt.plot(Ks, E_ins[i], label='$E_{in}, \lambda=$'+str(regs[i]))
plt.title('$E_{in}$ vs. K')
plt.xlabel('K')
plt.ylabel('Error')
plt.legend()
plt.savefig('2e_ein.png')	
plt.clf()

# Plot values of E_out across k for each value of lambda
for i in range(len(regs)):
    plt.plot(Ks, E_outs[i], label='$E_{out}, \lambda=$'+str(regs[i]))
plt.title('$E_{out}$ vs. K')
plt.xlabel('K')
plt.ylabel('Error')
plt.legend()	
plt.savefig('2e_eout.png')		


FileNotFoundError: train.txt not found.